In [1]:
import pandas as pd
import os

In [2]:
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = "data"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{file[0]}"
df_root = pd.read_excel(path)

In [9]:
df_root.activityState.unique()

array(['DATA_ENTRY_COMPLETE'], dtype=object)

In [4]:
df_root.Participante.unique()

array([11001], dtype=int64)

In [5]:
df_root.name.unique()

array(['Date of visit', 'Informed Consent', 'Demographics',
       'History of cutaneous leishmaniasis', 'Covid 19 testing',
       'Eligibility',
       'Medical Or Surgical History (other than Leishmaniasis)',
       'Vein assessment', 'Urinary Drug Screen', 'Alcohol Screen',
       'Clinical Laboratory - Test Hematology',
       'Clinical Laboratory Test - Clinical Chemistry',
       'Clinical Laboratory Test - Coagulation',
       'Immunoassay (Thyroid Stimulating Hormone)', 'Urinalysis',
       'Urine Microscopic Examination', 'Virology', 'Lesion Measurement',
       'Physical Examination', 'Vital Signs', '12-Lead ECG',
       'Clinical Laboratory Test - Clinical Chemistry D-1',
       'Interleukin-6', 'Titration Of Auto-Antibodies',
       'Injection Site Examination', 'PBMC Isolate',
       'Pharmacokinetic Blood Sampling (PK)',
       'Pharmacodynamic Blood Sampling (PD) -Cytokines/Chemokines',
       'mRNA Markers'], dtype=object)

In [5]:
pru = df_root[df_root['name']== '12-Lead ECG']

In [8]:
pru.T.head(50)

,197,198,199,200,201,202,203,204,205,273,...,348,349,350,351,352,353,354,355,356,357
Id Form,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
idCampo,630,632,635,636,637,638,639,640,641,630,...,700,701,702,703,704,705,706,707,708,709
Site,"01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia",...,"01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia","01 PECET/Clinica CES. Medellin, Colombia"
Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,Screening Visit,D-1,...,D1,D1,D1,D1,D1,D1,D1,D1,D1,D1
name,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,...,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG
Genero,Female,Female,Female,Female,Female,Female,Female,Female,Female,Female,...,Female,Female,Female,Female,Female,Female,Female,Female,Female,Female
AñoNacimiento,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
NameActividad,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,...,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG,12-Lead ECG
activityState,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,...,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE,DATA_ENTRY_COMPLETE
fecha enrolado,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,...,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46,2023-10-25 20:27:46


In [7]:
pru.Campo.unique()

array(['Was the ECG performed?', 'Date of ECG Performed',
       'Undefined, HR (bpm)', 'Undefined, RR (msec)',
       'Undefined, PR (msec)', 'Undefined, QRS (msec)',
       'Undefined, QT (msec)', 'Undefined, QTcF (msec)',
       'Undefined, Interpretation', 'Pre dose triplicate 1, Time 24 hrs',
       'Pre dose triplicate 1, HR (bpm)',
       'Pre dose triplicate 1, RR (msec)',
       'Pre dose triplicate 1, PR (msec)',
       'Pre dose triplicate 1, QRS (msec)',
       'Pre dose triplicate 1, QT (msec)',
       'Pre dose triplicate 1, QTcF (msec)',
       'Pre dose triplicate 1, Interpretation',
       'Pre dose triplicate 1, If abnormal, specify',
       'Pre dose triplicate 1, Please specify abnormal interpretation',
       'Pre dose triplicate 2, Time 24 hrs',
       'Pre dose triplicate 2, HR (bpm)',
       'Pre dose triplicate 2, RR (msec)',
       'Pre dose triplicate 2, PR (msec)',
       'Pre dose triplicate 2, QRS (msec)',
       'Pre dose triplicate 2, QT (msec)',
       

In [13]:
triplicate_1 = pru[pru['Campo']=='Pre dose triplicate 1, Time 24 hrs'].Valor.unique()[0]
triplicate_2 = pru[pru['Campo']=='Pre dose triplicate 2, Time 24 hrs'].Valor.unique()[0]
print(triplicate_1)
print(triplicate_2)

08:20
08:25


In [21]:
from datetime import datetime
time_format = '%H:%M'
triplicate_1_formated = datetime.strptime(triplicate_1, time_format)
triplicate_2_formated = datetime.strptime(triplicate_2, time_format)

time_difference = (triplicate_2_formated - triplicate_1_formated).total_seconds() / 60
print(time_difference)


5.0


In [12]:
import math
import numpy as np
prueba_variable = np.nan

if math.isnan(float(prueba_variable)):
    print('Si funciona asi')
else:
    print('No funciona asi') 

Si funciona asi


In [1]:
import math

if math.nan > 4:
    print('Si toca cambiar')
else:
    print('no toca cambiar')

no toca cambiar
